# overview
https://www.kaggle.com/jonathanbesomi/rsna-miccai-png/discussion/253022 によると、  
全ての患者に対して4種類の医療画像が用意されているわけではないらしい。  

用意されていないカテゴリファイルには空画像が格納されているとのこと。  
誤学習の要因となるため、画像の有無を明記したデータフレームを作成する。

# import dependancies

In [59]:
import os
import sys
from glob import glob
import pathlib
import re
import pandas as pd

# make pandas dataframe
このおじじが作ってくれたPNGデータセット  
https://www.kaggle.com/jonathanbesomi/rsna-miccai-png  
は、あらかじめ空画像の部分の画像ディレクトリは消去してあるので、  
このディレクトリ構造をそのままdataframeに書き写す。

まず本当にoriginal data と患者のidが変わっていないか確認

In [60]:
image_dirs = ([path for path in glob('../input/train/*') if re.search('\d{5}',str(path))])
pacient_ids = ([path.split('/')[-1] for path in image_dirs])
pacient_ids = pd.Series(pacient_ids, dtype='uint8') 

df = pd.read_csv('../input/train_labels.csv')
original_ids = df['BraTS21ID'].astype('uint8')

print(original_ids.equals(pacient_ids))

False


変わってない。おっけー  
ということでoriginal のdfに症例ごとのcolumnを作成し、画像の有無をboolean値として代入

In [55]:
image_types = ['FLAIR','T1w','T1wCE','T2w']
for image_type in image_types:
    df[image_type] = ''

for index,image_dir in enumerate(image_dirs):
    image_types_in_PNGdatasets = ([ image_type.split('/')[-1] for image_type in glob(os.path.join(image_dir,'*')) ] )
    for image_type in image_types:
        if image_type in image_types_in_PNGdatasets:
            df[image_type][index] = True
        else:
            df[image_type][index] = False

df.head(30)
            



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,BraTS21ID,MGMT_value,FLAIR,T1w,T1wCE,T2w
0,0,1,True,True,True,True
1,2,1,True,True,True,True
2,3,0,True,True,True,True
3,5,1,True,True,True,True
4,6,1,True,True,True,True
5,8,1,True,True,True,True
6,9,0,True,True,True,True
7,11,1,True,True,True,True
8,12,1,True,True,True,True
9,14,1,True,True,True,True


一応確認。ID　00109はFLAIRの写真だけないので、そこだけFalseになってるはず。

In [56]:
df.query('BraTS21ID == "109"')

,BraTS21ID,MGMT_value,FLAIR,T1w,T1wCE,T2w
71,109,1,False,True,True,True


csvに出力

In [57]:
df['BraTS21ID']=df['BraTS21ID'].map(lambda x:str(x).zfill(5))
print(df.head())
df.to_csv('../input/train_labels_add_imtype.csv',index=False)


  BraTS21ID  MGMT_value FLAIR   T1w T1wCE   T2w
0     00000           1  True  True  True  True
1     00002           1  True  True  True  True
2     00003           0  True  True  True  True
3     00005           1  True  True  True  True
4     00006           1  True  True  True  True


dataframeの[]内に条件を指定することで特定行を抜き出せる。  
以下のようにすると、FLAIRがTrueの行のみを抜き出したdataframeを作成可能。

In [58]:
temp_df = df[df["FLAIR"]==True] # ==Trueはなくてもおk
temp_df.query('BraTS21ID == "109"')

,BraTS21ID,MGMT_value,FLAIR,T1w,T1wCE,T2w
